# Agent Lanchain

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import re

# Tool: Extract the first email using the LLM
def extract_first_email_llm_tool(email_text):
    # Create a natural language prompt for the LLM
    extraction_prompt = f"""
    You are given a chain of emails. Your task is to extract the first email in the chain, 
    including the "From:", "To:", "Subject:", and the body of the email.
    
    Email chain:
    {email_text}
    
    Please extract the first email only.
    """
    
    return extraction_prompt

# Tool 1: Check if the recipient is from the same company
def check_recipient_tool(email_text):
    company_domain = "foyer.lu"
    recipient = re.search(r"To:.*?([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)", email_text)
    
    if recipient:
        email_address = recipient.group(1)
        if company_domain in email_address:
            return "The recipient is from the same company."
        else:
            return "The recipient is not from the same company."
    return "Could not find recipient information."

# Tool 2: Check if the email requires a reply
def check_reply_needed_tool(email_text):
    reply_keywords = ["reply", "respond", "please respond", "action needed", "awaiting your response"]
    if any(keyword in email_text.lower() for keyword in reply_keywords):
        return "The email requires a reply."
    else:
        return "The email does not require a reply."

# Tool 3: Check if the email is a reminder or recall
def check_reminder_or_recall_tool(email_text):
    reminder_keywords = ["reminder", "follow-up", "just a reminder", "recall", "as per my last email"]
    if any(keyword in email_text.lower() for keyword in reminder_keywords):
        return "The email is a reminder or recall."
    else:
        return "The email is not a reminder or recall."

# Load email text from a .txt file
def load_email_from_txt(file_path):
    with open(file_path, 'r') as file:
        email_text = file.read()
    return email_text

# List of tools to use in the agent
tools = [
    Tool(
        name="Extract First Email",
        func=extract_first_email_llm_tool,
        description="Use the LLM to extract the first email from a chain of emails."
    ),
    Tool(
        name="Check Recipient",
        func=check_recipient_tool,
        description="Check if the recipient is from the same company."
    ),
    Tool(
        name="Check Reply Needed",
        func=check_reply_needed_tool,
        description="Determine if the email requires a reply."
    ),
    Tool(
        name="Check Reminder/Recall",
        func=check_reminder_or_recall_tool,
        description="Check if the email is a reminder or a recall."
    )
]

# Initialize the agent using the tools and the OpenAI language model
llm = OpenAI(temperature=0)  # Replace with your preferred LLM
agent = initialize_agent(tools, llm, agent_type="zero-shot-react-description")

# Load email text from a .txt file (replace with your actual file path)
email_text = load_email_from_txt('/path/to/your/email.txt')

# Run the agent to extract the first email and analyze it
result = agent.run(email_text)
print(result)
